In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
# Menggunakan tanda seru (!) untuk menjalankan perintah shell
!git clone https://github.com/GreenAvo-Capstone/Machine-Learning.git

Cloning into 'Machine-Learning'...
remote: Enumerating objects: 15057, done.
remote: Counting objects: 100% (318/318), done.
remote: Compressing objects: 100% (298/298), done.
remote: Total 15057 (delta 18), reused 307 (delta 13), pack-reused 14739
Receiving objects: 100% (15057/15057), 412.15 MiB | 15.57 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Updating files: 100% (29426/29426), done.


In [3]:
import os

# Cek apakah direktori dataset sudah ada
dataset_dir = './Machine-Learning/dataset'
if os.path.exists(dataset_dir):
    print("Dataset berhasil diunduh.")
else:
    print("Dataset tidak ditemukan.")


Dataset berhasil diunduh.


In [4]:
path_train = os.path.join(dataset_dir, 'train')
path_val = os.path.join(dataset_dir, 'val')
path_test = os.path.join(dataset_dir, 'test')

In [5]:
BATCH_SIZE = 32
img_size = 224

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(path_train, shuffle=True, batch_size = BATCH_SIZE, image_size = (img_size, img_size), label_mode = "categorical")
valid_ds = tf.keras.utils.image_dataset_from_directory(path_val, shuffle=True, batch_size = BATCH_SIZE, image_size = (img_size, img_size), label_mode = "categorical")
test_ds = tf.keras.utils.image_dataset_from_directory(path_test, shuffle=False, batch_size = BATCH_SIZE, image_size = (img_size, img_size), label_mode = "categorical")

Found 11032 files belonging to 5 classes.
Found 2205 files belonging to 5 classes.
Found 1473 files belonging to 5 classes.


In [7]:
# Clear previous sessions
tf.keras.backend.clear_session()

# Define image size
img_size = 224

# Load datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_train,
    image_size=(img_size, img_size),
    batch_size=32,
    label_mode='int'
)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_val,
    image_size=(img_size, img_size),
    batch_size=32,
    label_mode='int'
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_test,
    image_size=(img_size, img_size),
    batch_size=32,
    label_mode='int'
)

# Get class names from training dataset
class_names = train_ds.class_names
num_class = len(class_names)

Found 11032 files belonging to 5 classes.
Found 2205 files belonging to 5 classes.
Found 1473 files belonging to 5 classes.


In [8]:
# Apply additional preprocessing if needed
def preprocess(image, label):
    image = tf.image.resize(image, [img_size, img_size])
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

    train_ds = train_ds.map(preprocess).prefetch(tf.data.experimental.AUTOTUNE)
    valid_ds = valid_ds.map(preprocess).prefetch(tf.data.experimental.AUTOTUNE)
    test_ds = test_ds.map(preprocess).prefetch(tf.data.experimental.AUTOTUNE)

# Check the shapes of a batch of images and labels
for images, labels in train_ds.take(1):
    print(f"Images shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")

Images shape: (32, 224, 224, 3)
Labels shape: (32,)


In [10]:
# Load MobileNetV3Large with pre-trained weights and make it trainable
base_model = tf.keras.applications.MobileNetV3Large(input_shape=(img_size, img_size, 3), include_top=False, pooling="max")
base_model.trainable = True

# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((img_size, img_size, 3), name="Input", dtype=tf.float32),
    tf.keras.layers.Rescaling(scale=1./255, name="Rescale_Layer"),
    base_model,
    tf.keras.layers.Dense(128, activation='selu', kernel_initializer="lecun_normal", name="FC"),
    tf.keras.layers.Dense(num_class, activation='softmax')
])

# Compile the model
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='precision'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='recall')
    ]
)

# Define callbacks
bestCB = tf.keras.callbacks.ModelCheckpoint(filepath=f'best_model.h5', verbose=1, save_best_only=True)
lrCB = tf.keras.callbacks.ReduceLROnPlateau(verbose=1, min_lr=1e-20, patience=5)
esCB = tf.keras.callbacks.EarlyStopping(patience=45, verbose=1)

12683000/12683000 [==============================] - 0s 0us/step


In [ ]:
history = model.fit(
    train_ds,
    validation_data = valid_ds,
    epochs = 50,
    callbacks=[bestCB, lrCB, esCB]
)

In [ ]:
model.save("./checkpoint/latest/")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label = "Training Loss")
plt.plot(history.history['val_loss'], label = "Validation Loss")
plt.legend(loc="best")
plt.title("Loss History")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

plt.plot(history.history['precision'], label = "Training Precision")
plt.plot(history.history['val_precision'], label = "Validation Precision")
plt.legend(loc="best")
plt.title("Precision History")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = history.epoch

plt.plot(epochs, acc, label='Train Accuracy')
plt.plot(epochs, val_acc, label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Model's Accuracy")
plt.legend()
plt.show()

plt.plot(epochs[10:], loss[10:], label='Train Loss')
plt.plot(epochs[10:], val_loss[10:], label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Model's Loss")
plt.legend()
plt.show()

In [ ]:
# Load iterasi terakhir
# model = tf.keras.models.load_model("./checkpoint/latest/")

# Load terbaik
best_model = tf.keras.models.load_model(f'best_model.h5')

# Evaluasi model pada data testing
loss, accuracy, precision, recall = best_model.evaluate(test_ds)
print("Loss pada data testing:", loss)
print("Akurasi pada data testing:", accuracy)
print("Precision pada data testing:", precision)
print("Recall pada data testing:", recall)

In [ ]:
# Prediksi kelas untuk setiap gambar di data testing
predictions = best_model.predict(test_ds)
predicted_classes = tf.math.argmax(predictions, axis=1)

In [ ]:
# Tampilkan beberapa contoh prediksi
for images, labels in test_ds.take(1):
    for i in range(5):  # Tampilkan 5 gambar pertama
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Prediksi: {class_names[predicted_classes[i]]}, \
                  Aktual: {class_names[labels[i]]}")
        plt.show()

In [ ]:
# Simpan Model
model.save('model_alpukat.h5')

# Convert the Model to TensorFlow Lite

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import tensorflow as tf

# Load model .h5
model = tf.keras.models.load_model('/content/best_model.h5')

# Konversi model ke format TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TFLite ke file
with open('Avocado Ripeness Model.tflite', 'wb') as f:
  f.write(tflite_model)